<a href="https://colab.research.google.com/github/Suchit027/PCAP-Lab/blob/main/lab%205/q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-02-01 19:03:06--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.32.29.16, 96.17.180.48
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.32.29.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   207MB/s    in 24s     

2025-02-01 19:03:31 (171 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]



In [6]:
%%writefile q3.cu
#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
__global__ void cal(double *a, int blocklimit, int threadlimit){
  int t = threadIdx.x;
  int b = blockIdx.x;
  if(b <= blocklimit && t <= threadlimit){
    int index = (b * threadlimit) + t;
    a[index] = sin(a[index]);
  }
  return;
}
int main(){
  double *a;
  int n;
  printf("enter n\n");
  scanf("%d", &n);
  a = (double *)malloc(n * sizeof(double));
  printf("enter the values\n");
  for(int i = 0; i < n; ++i){
    scanf("%lf", &a[i]);
  }
  double *da;
  cudaMalloc((void **)&da, n * sizeof(double));
  cudaMemcpy(da, a, n * sizeof(double), cudaMemcpyHostToDevice);
  int threadlimit;
  printf("enter threadlimit\n");
  scanf("%d", &threadlimit);
  int blocklimit;
  if (n % threadlimit == 0){
    blocklimit = n / threadlimit;
  }
  else{
    blocklimit = (n / threadlimit) + 1;
  }
  cal<<<blocklimit, 256>>>(da, blocklimit, threadlimit);
  cudaMemcpy(a, da, n * sizeof(double), cudaMemcpyDeviceToHost);
  for(int i = 0; i < n; ++i){
    printf("%lf ", a[i]);
  }
  return 0;
}

Overwriting q3.cu


In [7]:
!nvcc q3.cu -o q3.out

In [8]:
!./q3.out

enter n
3
enter the values
30 60 90
enter threadlimit
1
-0.988032 -0.304811 0.893997 